In [1]:
import requests
from abc import ABC, abstractmethod
from typing import List, Optional
import importlib
import backend.adapters.repository as repository
import backend.domain.model as model
import backend.service_layer.service as service
import backend.service_layer.uow as uow
from sec_api import XbrlApi
import os

importlib.reload(repository)
importlib.reload(model)
importlib.reload(service)

c:\Users\Site Oracle\reduceComplexity\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<module 'backend.service_layer.service' from 'c:\\Users\\Site Oracle\\reduceComplexity\\backend\\service_layer\\service.py'>

In [2]:
with uow.UnitOfWork() as uow_instance:
    res = service.get_consolidated_income_statements('aapl', uow_instance, form_type='10-K')

Time to get company: 1.03 seconds
Time to process filings: 13.36 seconds
Time to filter filings: 0.09 seconds
Time to create combined statements: 0.04 seconds

=== DEBUG: DATAFRAME MAPPING ===
DF1 index types: ['CostOfGoodsAndServicesSold', 'EarningsPerShareBasic', 'EarningsPerShareDiluted', 'GrossProfit', 'IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest', 'IncomeTaxExpenseBenefit', 'NetIncomeLoss', 'NonoperatingIncomeExpense', 'OperatingExpenses', 'OperatingIncomeLoss', 'ResearchAndDevelopmentExpense', 'RevenueFromContractWithCustomerExcludingAssessedTax', 'SellingGeneralAndAdministrativeExpense', 'WeightedAverageNumberOfDilutedSharesOutstanding', 'WeightedAverageNumberOfSharesOutstandingBasic']
DF2 index types: ['CommonStockDividendsPerShareDeclared', 'CostOfGoodsAndServicesSold', 'EarningsPerShareBasic', 'EarningsPerShareDiluted', 'GrossProfit', 'IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest', 

In [3]:
# Create a sample Apple instance using the UnitOfWork pattern
with uow.UnitOfWork() as uow_instance:
    apple = service.get_company_by_ticker('aapl', uow_instance)



In [4]:
apple.get_filings_by_type('10-K')

In [5]:
res.df

date_range,2021-09-26:2022-09-24,2022-09-25:2023-09-30,2023-10-01:2024-09-28,2011-09-25:2012-09-29,2012-09-30:2013-09-28,2013-09-29:2014-09-27,2018-09-30:2019-09-28,2019-09-29:2020-09-26,2020-09-27:2021-09-25,2015-09-27:2016-09-24,2016-09-25:2017-09-30,2017-10-01:2018-09-29,2014-09-28:2015-09-26
Cost of Goods Sold,2.235460e+11,2.141370e+11,2.103520e+11,8.784600e+10,1.066060e+11,1.122580e+11,1.617820e+11,1.695590e+11,2.129810e+11,1.313760e+11,1.410480e+11,1.637560e+11,1.400890e+11
Basic EPS,6.150000e+00,6.160000e+00,6.110000e+00,6.380000e+00,5.720000e+00,6.490000e+00,2.990000e+00,3.310000e+00,5.670000e+00,8.350000e+00,9.270000e+00,1.201000e+01,9.280000e+00
Diluted EPS,6.110000e+00,6.130000e+00,6.080000e+00,6.310000e+00,5.680000e+00,6.450000e+00,2.970000e+00,3.280000e+00,5.610000e+00,8.310000e+00,9.210000e+00,1.191000e+01,9.220000e+00
Gross Profit,1.707820e+11,1.691480e+11,1.806830e+11,6.866200e+10,6.430400e+10,7.053700e+10,9.839200e+10,1.049560e+11,1.528360e+11,8.426300e+10,8.818600e+10,1.018390e+11,9.362600e+10
Operating Income,1.191030e+11,1.137360e+11,1.234850e+11,5.576300e+10,5.015500e+10,5.348300e+10,6.573700e+10,6.709100e+10,1.092070e+11,6.137200e+10,6.408900e+10,7.290300e+10,7.251500e+10
Income Tax Expense,1.930000e+10,1.674100e+10,2.974900e+10,1.403000e+10,1.311800e+10,1.397300e+10,1.048100e+10,9.680000e+09,1.452700e+10,1.568500e+10,1.573800e+10,1.337200e+10,1.912100e+10
Net Income,9.980300e+10,9.699500e+10,9.373600e+10,4.173300e+10,3.703700e+10,3.951000e+10,5.525600e+10,5.741100e+10,9.468000e+10,4.568700e+10,4.835100e+10,5.953100e+10,5.339400e+10
Other Income/Expense,-3.340000e+08,-5.650000e+08,2.690000e+08,5.220000e+08,1.156000e+09,9.800000e+08,1.807000e+09,8.030000e+08,2.580000e+08,1.348000e+09,2.745000e+09,2.005000e+09,1.285000e+09
Operating Expenses,5.134500e+10,5.484700e+10,5.746700e+10,1.342100e+10,1.530500e+10,1.803400e+10,3.446200e+10,3.866800e+10,4.388700e+10,2.423900e+10,2.684200e+10,3.094100e+10,2.239600e+10
Operating Income,1.194370e+11,1.143010e+11,1.232160e+11,5.524100e+10,4.899900e+10,5.250300e+10,6.393000e+10,6.628800e+10,1.089490e+11,6.002400e+10,6.134400e+10,7.089800e+10,7.123000e+10


In [6]:
import repository

ModuleNotFoundError: No module named 'repository'

In [ ]:
indexes = repository.LLMRepository().make_index_readable(res.df.index.tolist())

In [ ]:
indexes

In [10]:
aapl = model.Company("aapl", "AAPL", repository.SECFilingRepository())

In [ ]:
aapl.filter_filings(model.FilingType.annual_report)

In [ ]:
generate_text("Hello, how are you?")

In [38]:
# Create a function to use Google's Gemini model for text generation
import google.generativeai as genai
import os
from dotenv import load_dotenv

def setup_gemini():
    load_dotenv()
    api_key = os.getenv("GEMINI_API_KEY")
    if not api_key:
        raise ValueError("GEMINI_API_KEY environment variable not set")
    
    genai.configure(api_key=api_key)
    # Use a model that exists in the current API version
    return genai.GenerativeModel('gemini-1.5-pro')

def generate_text(prompt, model=None):
    if model is None:
        model = setup_gemini()
    
    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        print(f"Error generating text: {e}")
        return None


In [16]:
def get_annual_filing(aapl, index):
    filings = [x for x in aapl.filings if x.form == "10-K"]
    return filings[index]
        

In [ ]:
filings = [x for x in aapl.filings if x.form == "10-K"]

In [ ]:
# Merge all annual filing income statements into a single dataframe
import pandas as pd

all_dfs = []
for i, filing in enumerate(filings):
    print(filing.cik)
    print(filing.accession_number)
    print(filing.primary_document)
    print(filing.filing_date)
    print(filing.form)
    df = filing.income_statement.table
    if not df.empty:
        all_dfs.append(df)
        print(f"Added filing {i+1}: {filing.filing_date}")

# Merge all dataframes
if all_dfs:
    merged_df = pd.concat(all_dfs, axis=1)
    print(f"Successfully merged {len(all_dfs)} income statements")
    df = merged_df
else:
    print("No valid income statements found")


In [19]:
# Remove duplicate columns
df_no_dupes = merged_df.loc[:, ~merged_df.columns.duplicated()]

# Sort columns by date (earliest to latest)
sorted_columns = sorted(df_no_dupes.columns)
df_no_dupes = df_no_dupes[sorted_columns]

In [ ]:
df_no_dupes

In [ ]:
filing.income_statement.table

In [ ]:
df.columns

In [ ]:
df